
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 2.3 DEMO: Implementing Delta Sharing (Databricks-to-Open)

## Overview
In this demo, we will implement Delta Sharing to securely share data with non-Databricks recipients using **Open Sharing**. This demonstration shows how to share data with external partners who don't have access to a Databricks workspace.

**Provider Notebook (This Notebook):** We will use the existing Unity Catalog data from the previous demo, create a share for external access, configure an open recipient using bearer token authentication, and generate an activation link and credential file for the recipient.

**Recipient Notebook:** The receiving organization (not using Databricks) will download the credential file and access the shared data using various tools including Python, pandas, Power BI, and other compatible platforms.

This demo showcases the **Databricks-to-Open (D2O)** sharing pattern, where the provider uses Databricks and the recipient uses open-source or third-party tools to access the data.

### Learning Objectives
By the end of this demo, you will be able to:
1. Create a recipient for open sharing using bearer token authentication
2. Understand the difference between D2D and D2O sharing patterns
3. Generate and share activation links and credential files securely
4. Understand alternative authentication methods (OIDC federation)
5. Manage recipient tokens and security considerations
6. Grant non-Databricks recipients access to shared data

## Background

**Scenario:**
You are a data provider company (*"Acme Corp"*) that wants to share customer analytics data with an external partner organization that does not have access to Databricks. The partner will access the shared data using Python, Power BI, or other compatible tools.

### Open Sharing Authentication Methods

Databricks supports two authentication flows for open sharing:

1. **Bearer Token Authentication** (demonstrated in this notebook)
   - Provider creates recipient and Databricks generates a long-lived bearer token
   - Token is included in a credential file shared via activation link
   - Simple to set up but requires careful token management
   - Recipient treats credential file as a secret

2. **OIDC Federation** (mentioned for awareness)
   - Uses OpenID Connect to federate authentication to recipient's Identity Provider (IdP)
   - Supports both User-to-Machine (U2M) and Machine-to-Machine (M2M) flows
   - More secure: uses short-lived JWT tokens instead of long-lived bearer tokens
   - Enables MFA and fine-grained access control
   - IdP manages token lifecycle and security policies
   - Recommended for enterprise scenarios requiring enhanced security

For this demo, we'll use bearer token authentication for simplicity. In production environments, consider OIDC federation for enhanced security.

In [0]:
%run ./Includes/Demo-Setup-2

## Step 1: Review Existing Data

We'll use the same catalog and data created in the previous demo (D2D). Let's verify the data is available.

In [0]:
-- Verify current context
SELECT current_catalog(), current_schema();

In [0]:
-- Show available tables
SHOW TABLES;

## Step 2: Create a Share for External Access

We'll create a new share specifically for our external partner. This allows us to manage internal and external sharing separately.

In [0]:
-- Create a share for external partners
CREATE SHARE IF NOT EXISTS external_retail
COMMENT 'Share containing customer and sales data for external partner access';

In [0]:
-- Add tables to the external share
ALTER SHARE external_retail 
ADD TABLE acme_corp.retail.customers
COMMENT 'Customer master data for external partners';

ALTER SHARE external_retail 
ADD TABLE acme_corp.retail.sales_transactions
COMMENT 'Sales transaction history for external partners';

In [0]:
-- Verify share contents
SHOW ALL IN SHARE external_retail;

## Step 3: Create an Open Recipient (Bearer Token)

For open sharing, we create a recipient without specifying a metastore ID. Databricks will automatically:
1. Set the authentication type to `TOKEN`
2. Generate a bearer token
3. Create a credential file containing the token
4. Generate an activation link for downloading the credential file

**Important Security Considerations:**
- The credential file can only be downloaded once via the activation link
- Treat the activation link and credential file as secrets
- Share the activation link over a secure channel only
- Recipients must not share credentials outside their organization
- You can rotate tokens if credentials are compromised

In [0]:
-- Create an open recipient (no metastore ID required)
CREATE RECIPIENT IF NOT EXISTS partner_co
COMMENT 'External partner organization - open sharing recipient';

In [0]:
-- View all recipients (note the different authentication types)
SHOW RECIPIENTS;

## Step 4: Get the Activation Link

The activation link allows the recipient to download their credential file. **This link should be shared securely** (e.g., encrypted email, secure messaging platform).

**Important Notes:**
- The activation link is only displayed if the credential file has not been downloaded yet
- After the recipient downloads the credential file, the activation link will no longer be shown
- If you need to provide the credential again, you must rotate the recipient's token

In [0]:
-- Get the activation link for the recipient
DESCRIBE RECIPIENT partner_co;

**Action Required:** Copy the `activation_link` from the output above and share it securely with your recipient.

The activation link format will look like:
```
https://<workspace-url>/accept-credentials?shareId=<id>&activationToken=<token>
```

## Step 5: Grant the Recipient Access to the Share

Now we grant the open recipient access to our external share.

In [0]:
-- Grant the open recipient access to the share
GRANT SELECT ON SHARE external_retail TO RECIPIENT partner_co;

In [0]:
-- Verify the grant
SHOW GRANTS ON SHARE external_retail;

In [0]:
-- Check all shares the recipient has access to
SHOW GRANTS TO RECIPIENT partner_co;

## Step 6: Managing Recipient Tokens

As a provider, you can manage recipient tokens for security purposes.

### When to Rotate Tokens

You should rotate a recipient's token in these circumstances:
- The existing token is about to expire
- A recipient loses their activation URL or it's compromised
- The credential file is lost or compromised after download
- When modifying the recipient token lifetime for the metastore
- If the activation URL was sent over an insecure channel or to the wrong person

### Security Best Practices

- **Set token expiration**: Configure tokens to expire after a reasonable period
- **Rotate compromised tokens immediately**: Set existing token expiration to 0 seconds
- **Use secure channels**: Always share activation links through encrypted communications
- **Monitor access**: Regularly review audit logs for unusual access patterns
- **Document sharing agreements**: Maintain records of what data is shared with whom

### Token Rotation Process (UI)

To rotate a token in the UI:
1. Go to Catalog → Delta Sharing → Shared by me → Recipients
2. Select the recipient
3. On the Details tab, under Token Expiration, click **Rotate**
4. Set expiration time (immediately or specific duration)
5. Copy the new activation link and share it securely

**Note:** You can have at most two tokens active at once (an active token and a rotated token awaiting expiration).

## Step 7: Optional IP Access Lists

For enhanced security, you can restrict recipient access to specific IP addresses. This is useful when:
- The recipient accesses data from known, fixed IP addresses
- You want to add an extra layer of security beyond bearer tokens
- Compliance requirements mandate IP restrictions

IP access lists can be configured:
- When creating the recipient
- Added to an existing recipient
- Modified as needed

See the Databricks documentation for details on configuring IP access lists for Delta Sharing recipients.

## Alternative: OIDC Federation (Advanced)

For production environments requiring enhanced security, consider **OIDC Federation** instead of bearer tokens:

### Benefits of OIDC Federation
- **Short-lived tokens**: JWT tokens have limited lifetime (managed by recipient's IdP)
- **No credential management**: Databricks doesn't generate or manage tokens
- **MFA support**: Recipient's IdP can enforce multi-factor authentication
- **Fine-grained control**: Define access by specific users, groups, or service principals
- **Audit trail**: Authentication events logged in recipient's IdP

### Authentication Flows

**User-to-Machine (U2M)**:
- End users authenticate via their IdP (e.g., Microsoft Entra ID, Okta)
- Used with tools like Power BI, Tableau
- OAuth Authorization Code Grant flow
- MFA enforced by IdP

**Machine-to-Machine (M2M)**:
- Service principals or OAuth applications authenticate
- Used for automated workloads (nightly jobs, background services)
- OAuth Client Credentials Grant flow
- No user interaction required

### OIDC Federation Setup (High-Level)

1. **Create Open OIDC Federation Recipient**:
   - In Catalog Explorer → Delta Sharing → New recipient
   - Recipient type: Open
   - Authentication method: OIDC Federation

2. **Configure OIDC Policy**:
   - Policy name
   - Issuer URL (e.g., `https://login.microsoftonline.com/{tenant-id}/v2.0`)
   - Subject claim (e.g., `oid` for users, `groups` for groups, `azp` for M2M apps)
   - Subject (user/group/app ID)
   - Audiences (resource identifiers)

3. **Share OIDC Profile**:
   - Databricks generates an OIDC profile web portal URL
   - Share URL with recipient (contains no sensitive information)
   - Recipient downloads profile or copies endpoint URL

4. **Recipient Authentication**:
   - Recipient authenticates via their IdP
   - IdP issues JWT with identity claims
   - Databricks validates JWT against the configured policy
   - If valid, access is granted

For detailed OIDC Federation implementation, refer to the Databricks documentation.

## Next Steps

The recipient will now:
1. Access the activation link (shared securely)
2. Download the credential file (`.share` format)
3. Use the credential file to access data via:
   - Python (`delta-sharing` library)
   - pandas (via `delta-sharing` connector)
   - Apache Spark
   - Power BI Desktop
   - Tableau
   - Other Delta Sharing compatible tools

See the recipient notebook for detailed instructions on accessing shared data.

## Summary

✅ **What we accomplished:**

1. Created a Delta Share for external partner access
2. Created an open recipient using bearer token authentication
3. Generated an activation link and credential file
4. Granted the recipient access to the share
5. Learned about token management and security considerations
6. Explored alternative OIDC federation authentication

**Key Differences: D2D vs D2O**
- **D2D (Databricks-to-Databricks)**:
  - Recipient has Databricks workspace
  - Authentication via metastore ID
  - Mount share directly to catalog
  - Query with standard SQL

- **D2O (Databricks-to-Open)**:
  - Recipient doesn't need Databricks
  - Authentication via bearer token or OIDC
  - Use credential file with various tools
  - Query with Python, Power BI, etc.

**Key Benefits:**
- 🚀 **No Platform Lock-in**: Recipients use their preferred tools
- 🔒 **Secure**: Token-based or OIDC authentication
- ⚡ **Real-time**: Always access latest data from provider
- 💰 **Cost-effective**: No storage costs for recipients
- 🌐 **Open Standard**: Delta Sharing is an open protocol

---
&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>